In [1]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
import pandas as pd

df = pd.read_csv('cleaned.csv')
df = df.drop(columns="quality")
print(df.columns.values)
print(df.shape)
df.head()

['fixed_acidity' 'volatile_acidity' 'citric_acid' 'residual_sugar'
 'chlorides' 'free_sulfur_dioxide' 'total_sulfur_dioxide' 'density' 'pH'
 'sulphates' 'alcohol' 'good_quality_1']
(4898, 12)


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,good_quality_1
0,0.172079,-0.081762,0.213258,2.821061,-0.035351,0.569873,0.744489,2.331274,-1.246794,-0.349149,-1.393010,0
1,-0.657434,0.215874,0.047996,-0.944669,0.147732,-1.252891,-0.149669,-0.009153,0.739953,0.001342,-0.824192,0
2,1.475600,0.017450,0.543783,0.100272,0.193503,-0.312109,-0.973236,0.358628,0.475053,-0.436771,-0.336633,0
3,0.409083,-0.478608,-0.117266,0.415726,0.559670,0.687471,1.120977,0.525802,0.011479,-0.787262,-0.499152,0
4,0.409083,-0.478608,-0.117266,0.415726,0.559670,0.687471,1.120977,0.525802,0.011479,-0.787262,-0.499152,0


In [3]:
df.groupby("good_quality_1").count()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
good_quality_1,,,,,,,,,,,
0,3838,3838,3838,3838,3838,3838,3838,3838,3838,3838,3838
1,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060


In [4]:
from sklearn.model_selection import train_test_split

df4partition = df.copy()
testpart_size = 0.2

df_nontestData, df_testData = train_test_split(df4partition, test_size = testpart_size, random_state = 1)
print(df_nontestData)

      fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
3749      -0.657434          0.017450    -0.365159        0.080556   0.239274   
1344       0.527585         -0.478608     0.295890       -0.885521  -0.218435   
58        -0.301928          1.009567    -1.521995       -0.353193  -0.081122   
3177      -1.486947          1.307202    -0.943577        0.139704   0.193503   
3942       0.409083         -0.478608     0.461152       -0.964385   0.330815   
...             ...               ...          ...             ...        ...   
2895      -0.538932          1.604838    -0.613053       -0.865805   0.376586   
2763       1.001593          0.215874     0.047996       -1.023532   0.101961   
905        1.831106         -0.875455     0.709045       -0.944669   0.056190   
3980      -0.894438         -0.577820    -0.695684        1.125497  -0.493060   
235        0.409083         -0.478608     0.378521        1.559246   0.559670   

      free_sulfur_dioxide  

In [5]:
from sklearn.neighbors import KNeighborsClassifier

DV = 'good_quality_1'
y = df_nontestData[DV]
X = df_nontestData.drop(columns = [DV])

k = 5

clf = KNeighborsClassifier(metric='euclidean', n_neighbors=k).fit(X,y)

X_test = df_testData.drop(columns=DV)
y_test = df_testData[DV]

from sklearn import metrics

model_object = clf

print(metrics.roc_auc_score(y_test, model_object.predict_proba(X_test)[:, 1]))

0.8371159956837705


## AUC

In [6]:
kfolds = 5
max_k = 200

param_grid = {'n_neighbors': list(range(1, max_k+1))}

from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(KNeighborsClassifier(metric='euclidean'), param_grid, scoring='roc_auc', cv=kfolds, n_jobs=-1)
gridsearch.fit(X,y)
clf_bestkNN = gridsearch.best_estimator_

In [7]:
print(clf_bestkNN.n_neighbors)

y_test_actual = df_testData[DV]

X_test = df_testData.drop(columns=[DV])

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test_actual, clf_bestkNN.predict_proba(X_test)[:,1]))

49
0.837974472245572


In [8]:
clf_bestkNN.predict_proba(X_test)[:,1]

array([0.06122449, 0.        , 0.28571429, 0.        , 0.65306122,
       0.        , 0.        , 0.04081633, 0.14285714, 0.04081633,
       0.79591837, 0.10204082, 0.48979592, 0.        , 0.36734694,
       0.18367347, 0.36734694, 0.10204082, 0.04081633, 0.10204082,
       0.28571429, 0.18367347, 0.32653061, 0.34693878, 0.30612245,
       0.20408163, 0.34693878, 0.        , 0.3877551 , 0.44897959,
       0.04081633, 0.44897959, 0.30612245, 0.08163265, 0.55102041,
       0.08163265, 0.        , 0.10204082, 0.02040816, 0.3877551 ,
       0.3877551 , 0.18367347, 0.30612245, 0.06122449, 0.14285714,
       0.        , 0.34693878, 0.02040816, 0.14285714, 0.34693878,
       0.42857143, 0.24489796, 0.14285714, 0.18367347, 0.3877551 ,
       0.63265306, 0.30612245, 0.34693878, 0.2244898 , 0.20408163,
       0.42857143, 0.12244898, 0.30612245, 0.10204082, 0.32653061,
       0.73469388, 0.12244898, 0.30612245, 0.42857143, 0.10204082,
       0.12244898, 0.04081633, 0.32653061, 0.        , 0.12244

## Accuracy

In [9]:
kfolds = 5
max_k = 200

param_grid = {'n_neighbors': list(range(1, max_k+1))}

from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(KNeighborsClassifier(metric='euclidean'), param_grid, scoring='accuracy', cv=kfolds, n_jobs=-1)
gridsearch.fit(X,y)
clf_bestkNN = gridsearch.best_estimator_

In [10]:
print(clf_bestkNN.n_neighbors)

y_test_actual = df_testData[DV]

X_test = df_testData.drop(columns=[DV])

# from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_test_actual, clf_bestkNN.predict_proba(X_test)[:,1]))

# Import the metrics package
from sklearn import metrics

# Display the confusion matrix over the test partition
metrics.confusion_matrix(y_test_actual, clf_bestkNN.predict(X_test))

2


array([[731,  28],
       [119, 102]], dtype=int64)

In [11]:
clf_bestkNN.score(X_test, y_test_actual )

0.85